<a href="https://colab.research.google.com/github/Mansiraj1309/Mansiraj1309/blob/main/Image_captioning_in_hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install indic_nlp_library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 10.3 MB/s eta 0:00:00


In [4]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import re
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import layers
from keras.applications import efficientnet
from keras.layers import TextVectorization

from indicnlp.tokenize import indic_tokenize  # Import indic_tokenize


keras.utils.set_random_seed(111)

In [5]:

# Path to the images
IMAGES_PATH = "hindi_dataset"

# Desired image dimensions
IMAGE_SIZE = (299, 299)

# Vocabulary size
VOCAB_SIZE = 10000

# Fixed length allowed for any sequence
SEQ_LENGTH = 25

# Dimension for the image embeddings and token embeddings
EMBED_DIM = 512

# Per-layer units in the feed-forward network
FF_DIM = 512

# Other training parameters
BATCH_SIZE = 64
EPOCHS = 30
AUTOTUNE = tf.data.AUTOTUNE

In [6]:
ls

 drive/   hindi_dataset.txt   sample_data/  'Untitled Folder'/


In [7]:
from itertools import islice

In [8]:
from indicnlp.tokenize import indic_tokenize  # Import indic_tokenize

def load_captions_data(filename):

    """Loads captions (text) data and maps them to corresponding images.

    Args:
        filename: Path to the text file containing caption data.

    Returns:
        caption_mapping: Dictionary mapping image names and the corresponding captions
        text_data: List containing all the available captions
    """
    caption_mapping = {}
    text_data = []
    images_to_skip = set()


    with open(filename) as caption_file:
        for line in caption_file:
            line = line.rstrip("\n")
            # Check if the line contains a tab character before splitting
            if "\t" in line:
                img_name, caption = line.split("\t")

                # Each image is repeated five times for the five different captions.
                # Each image name has a suffix `#(caption_number)`
                img_name = img_name.split("#")[0]
                img_name = os.path.join(IMAGES_PATH, img_name.strip())

                # We will remove caption that are either too short to too long
                tokens = caption.strip().split()

                if len(tokens) < 5 or len(tokens) > SEQ_LENGTH:
                    images_to_skip.add(img_name)
                    continue

                if img_name.endswith("jpg") and img_name not in images_to_skip:
                    # We will add a start and an end token to each caption
                    caption = "<start> " + caption.strip() + " <end>"
                    text_data.append(caption)

                    if img_name in caption_mapping:
                        caption_mapping[img_name].append(caption)
                    else:
                        caption_mapping[img_name] = [caption]
            else:
                # Handle lines without a tab character (e.g., skip or print a warning)
                print(f"Skipping line: {line} - Does not contain a tab character.")

    for img_name in images_to_skip:
        if img_name in caption_mapping:
            del caption_mapping[img_name]

    return caption_mapping, text_data

def train_val_split(caption_data, train_size=0.8, shuffle=True):
    """Split the captioning dataset into train and validation sets.

    Args:
        caption_data (dict): Dictionary containing the mapped caption data
        train_size (float): Fraction of all the full dataset to use as training data
        shuffle (bool): Whether to shuffle the dataset before splitting

    Returns:
        Traning and validation datasets as two separated dicts
    """

    # 1. Get the list of all image names
    all_images = list(caption_data.keys())

    # 2. Shuffle if necessary
    if shuffle:
        np.random.shuffle(all_images)

    # 3. Split into training and validation sets
    train_size = int(len(caption_data) * train_size)

    training_data = {
        img_name: caption_data[img_name] for img_name in all_images[:train_size]
    }
    validation_data = {
        img_name: caption_data[img_name] for img_name in all_images[train_size:]
    }

    # 4. Return the splits
    return training_data, validation_data


# Load the dataset
captions_mapping, text_data = load_captions_data("hindi_dataset.txt")

# Split the dataset into training and validation sets
train_data, valid_data = train_val_split(captions_mapping)
print("Number of training samples: ", len(train_data))
print("Number of validation samples: ", len(valid_data))

Streaming output truncated to the last 5000 lines.
Skipping line: 378453580_21d688748e,कुत्ते छड़ी पर छड़ी और कूदता है। - Does not contain a tab character.
Skipping line: 378453580_21d688748e,कुत्ता छड़ी ले जाता है और जंगल में लॉग पर कूदता है। - Does not contain a tab character.
Skipping line: 378453580_21d688748e,कुत्ता अपने मुंह में छड़ी के साथ लॉग पर कूदता है। - Does not contain a tab character.
Skipping line: 379006645_b9a2886b51,काले और भूरे रंग का कुत्ता बर्फ के साथ सीमेंट बाधाओं के बीच चल रहा है। - Does not contain a tab character.
Skipping line: 379006645_b9a2886b51,कुत्ता ठंडी गलियों में भागता है। - Does not contain a tab character.
Skipping line: 379006645_b9a2886b51,काले और तन छोटे कुत्ते पर्कड कान के साथ चलते हैं। - Does not contain a tab character.
Skipping line: 379006645_b9a2886b51,काले और भूरे रंग के कुत्ते संलग्न बर्फीले क्षेत्र में कैमरे की ओर चलते हैं। - Does not contain a tab character.
Skipping line: 379006645_b9a2886b51,भूरा और काला कुत्ता बर्फीली गली से गुजर रहा 

In [9]:
captions_mapping, text_data=load_captions_data("hindi_dataset.txt")

Streaming output truncated to the last 5000 lines.
Skipping line: 378453580_21d688748e,कुत्ता लकड़ी के क्षेत्र में लॉग पर कूद रहा है जबकि दूसरा लॉग ले जा रहा है। - Does not contain a tab character.
Skipping line: 378453580_21d688748e,जंगल में कुत्ते का डंडा जंगल में गिरे पेड़ पर कूदता है। - Does not contain a tab character.
Skipping line: 378453580_21d688748e,कुत्ते छड़ी पर छड़ी और कूदता है। - Does not contain a tab character.
Skipping line: 378453580_21d688748e,कुत्ता छड़ी ले जाता है और जंगल में लॉग पर कूदता है। - Does not contain a tab character.
Skipping line: 378453580_21d688748e,कुत्ता अपने मुंह में छड़ी के साथ लॉग पर कूदता है। - Does not contain a tab character.
Skipping line: 379006645_b9a2886b51,काले और भूरे रंग का कुत्ता बर्फ के साथ सीमेंट बाधाओं के बीच चल रहा है। - Does not contain a tab character.
Skipping line: 379006645_b9a2886b51,कुत्ता ठंडी गलियों में भागता है। - Does not contain a tab character.
Skipping line: 379006645_b9a2886b51,काले और तन छोटे कुत्ते पर्कड कान के साथ

In [10]:

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


strip_chars = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
strip_chars = strip_chars.replace("<", "")
strip_chars = strip_chars.replace(">", "")

vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=SEQ_LENGTH,
    standardize=custom_standardization,
)
vectorization.adapt(text_data)
vectorization.get_vocabulary()

# Data augmentation for image data
image_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomContrast(0.3),
    ]
)


In [3]:
!pip uninstall indic-nlp-library -y
!pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 10.7 MB/s eta 0:00:00


In [4]:
!find / -name indicnlp 2>/dev/null # replace / with the base environment path if known

/usr/local/lib/python3.11/dist-packages/indicnlp


In [5]:
try:
    import indicnlp
    print("indicnlp is installed.")
except ImportError:
    print("indicnlp is not installed. Please install it using: !pip install indic-nlp-library")

indicnlp is installed.


In [6]:
try:
    import fasttext
    print("fasttext is installed.")
except ImportError:
    print("fasttext is not installed. Please install it using: !pip install fasttext")

fasttext is not installed. Please install it using: !pip install fasttext


In [24]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313475 sha256=de1deb96729ce77d060de485755314aa0de059294f46b038b82f3b9a14037500
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [26]:
!pip install tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [38]:
import os
import tensorflow as tf
# Define IMAGES_PATH here
IMAGES_PATH = "/content/sample_data"  # Corrected path to your images directory
# 1. Call load_captions_data to initialize captions_mapping
IMAGE_SIZE = (299, 299)
SEQ_LENGTH = 25

from indicnlp.tokenize import indic_tokenize  # Ensure indic_tokenize is available

def load_captions_data(filename):
    """Loads captions (text) data and maps them to corresponding images.

    Args:
        filename: Path to the text file containing caption data.

    Returns:
        caption_mapping: Dictionary mapping image names and the corresponding captions
        text_data: List containing all the available captions
    """
    caption_mapping = {}
    text_data = []
    images_to_skip = set()

    # Get the absolute path of the current file
    # Construct the full path to the dataset file
    full_path = os.path.join(IMAGES_PATH, filename) # If hindi_dataset.txt is in the same directory as the notebook.
    # OR
    # full_path = "path/to/your/data/hindi_dataset.txt" # If the path to the file is known


    with open(full_path) as caption_file:
        for line in caption_file:
            line = line.rstrip("\n")
            # Check if the line contains a tab character before splitting
            if "\t" in line:
                img_name, caption = line.split("\t")

                # Each image is repeated five times for the five different captions.
                # Each image name has a suffix `#(caption_number)`
                img_name = img_name.split("#")[0]
                img_name = os.path.join(IMAGES_PATH, img_name.strip())

                # We will remove caption that are either too short to too long
                tokens = caption.strip().split()

                if len(tokens) < 5 or len(tokens) > SEQ_LENGTH:
                    images_to_skip.add(img_name)
                    continue

                if img_name.endswith("jpg") and img_name not in images_to_skip:
                    # We will add a start and an end token to each caption
                    caption = "<start> " + caption.strip() + " <end>"
                    text_data.append(caption)

                    if img_name in caption_mapping:
                        caption_mapping[img_name].append(caption)
                    else:
                        caption_mapping[img_name] = [caption]
            else:
                # Handle lines without a tab character (e.g., skip or print a warning)
                print(f"Skipping line: {line} - Does not contain a tab character.")

    for img_name in images_to_skip:
        if img_name in caption_mapping:
            del caption_mapping[img_name]

    return caption_mapping, text_data  # This line is crucial to return the data

captions_mapping = {os.path.join(IMAGES_PATH, os.path.basename(k)): v
                    for k, v in captions_mapping.items()}

captions_mapping, text_data = load_captions_data("/content/sample_data/hindi_dataset.txt")

# Get a list of all image filenames in the IMAGES_PATH directory
image_filenames = [os.path.join(IMAGES_PATH, filename) for filename in os.listdir(IMAGES_PATH) if filename.endswith(".jpg")]
# ... (Rest of your code remains the same)
# ... (Rest of your code remains the same)

# Extract captions corresponding to the image filenames
captions = [caption for img_name, captions_list in captions_mapping.items()
               for caption in captions_list if img_name in image_filenames]
# ... (Rest of your code remains the same)
# Define function to decode and resize the images
def decode_and_resize(img_path):
    img_path = img_path.decode() if isinstance(img_path, bytes) else str(img_path)

    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

# Define function to vectorize and pad the captions
def vectorize_captions(captions):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(captions)  # Fit the tokenizer on the captions
    # Tokenize the captions
    tokenized_captions = tokenizer.texts_to_sequences(captions)
    # Pad the tokenized captions to the same length
    padded_captions = pad_sequences(tokenized_captions, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    return padded_captions

# Define function to process input: Image and captions
def process_input(img_path, captions):
    img = decode_and_resize(img_path)
    padded_captions = vectorize_captions(captions)
    return img, padded_captions

# Define function to create the dataset
def make_dataset(images, captions, BATCH_SIZE=64, AUTOTUNE=tf.data.AUTOTUNE):
    """Creates a tf.data dataset for training or validation.
    """
    # Ensure images are strings before creating the dataset
    images = [str(image) if not isinstance(image, str) else image for image in images]
    # 1. Vectorize captions *outside* the dataset pipeline
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(captions)

    def vectorize_caption(caption):
        sequence = tokenizer.texts_to_sequences([caption.numpy().decode() if isinstance(caption, bytes) else caption])[0]  # Decode if necessary
        return tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=SEQ_LENGTH, padding='post')[0]

    dataset = tf.data.Dataset.from_tensor_slices((images, captions))

    # Shuffle, map, batch, and prefetch
    dataset = dataset.shuffle(BATCH_SIZE * 8)
    # 2. Apply vectorization using tf.py_function
    dataset = dataset.map(lambda img, cap: (decode_and_resize(img), tf.py_function(vectorize_caption, [cap], tf.int32)),
                         num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return dataset


# Step 4: Create the train and validation datasets
# For training data
train_images = image_filenames  # List of image filenames
train_captions = captions  # Corresponding captions
train_dataset = make_dataset(train_images, train_captions) # Call make_dataset with default BATCH_SIZE and AUTOTUNE

# ... [Rest of your code]

# For validation data (if you have a separate validation dataset, use the same structure)
valid_images = image_filenames  # List of image filenames (use separate validation data if available)
valid_captions = captions  # Corresponding captions (use separate validation captions if available)
valid_dataset = make_dataset(valid_images, valid_captions)

# Step 5: Inspect a batch of the dataset
for image, caption in train_dataset.take(1):
    print("Image shape:", image.shape)
    print("Caption shape:", caption.shape)
    print("First caption (as tokens):", caption[0])

Streaming output truncated to the last 5000 lines.
Skipping line: 378453580_21d688748e,कुत्ता लकड़ी के क्षेत्र में लॉग पर कूद रहा है जबकि दूसरा लॉग ले जा रहा है। - Does not contain a tab character.
Skipping line: 378453580_21d688748e,जंगल में कुत्ते का डंडा जंगल में गिरे पेड़ पर कूदता है। - Does not contain a tab character.
Skipping line: 378453580_21d688748e,कुत्ते छड़ी पर छड़ी और कूदता है। - Does not contain a tab character.
Skipping line: 378453580_21d688748e,कुत्ता छड़ी ले जाता है और जंगल में लॉग पर कूदता है। - Does not contain a tab character.
Skipping line: 378453580_21d688748e,कुत्ता अपने मुंह में छड़ी के साथ लॉग पर कूदता है। - Does not contain a tab character.
Skipping line: 379006645_b9a2886b51,काले और भूरे रंग का कुत्ता बर्फ के साथ सीमेंट बाधाओं के बीच चल रहा है। - Does not contain a tab character.
Skipping line: 379006645_b9a2886b51,कुत्ता ठंडी गलियों में भागता है। - Does not contain a tab character.
Skipping line: 379006645_b9a2886b51,काले और तन छोटे कुत्ते पर्कड कान के साथ

In [41]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import re
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import layers
from keras.applications import efficientnet
from keras.layers import TextVectorization

from indicnlp.tokenize import indic_tokenize  # Import indic_tokenize


keras.utils.set_random_seed(111)

# Path to the images
IMAGES_PATH = "hindi_dataset"

# Desired image dimensions
IMAGE_SIZE = (299, 299)

# Vocabulary size
VOCAB_SIZE = 10000

# Fixed length allowed for any sequence
SEQ_LENGTH = 25

# Dimension for the image embeddings and token embeddings
EMBED_DIM = 512

# Per-layer units in the feed-forward network
FF_DIM = 512

# Other training parameters
BATCH_SIZE = 64
EPOCHS = 30
AUTOTUNE = tf.data.AUTOTUNE

def get_cnn_model():
    base_model = efficientnet.EfficientNetB0(
        input_shape=(*IMAGE_SIZE, 3),
        include_top=False,
        weights="imagenet",
    )
    # We freeze our feature extractor
    base_model.trainable = False
    base_model_out = base_model.output
    base_model_out = layers.Reshape((-1, base_model_out.shape[-1]))(base_model_out)
    cnn_model = keras.models.Model(base_model.input, base_model_out)
    return cnn_model



class TransformerEncoderBlock(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.0
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.dense_1 = layers.Dense(embed_dim, activation="relu")

    def call(self, inputs, training, mask=None):
        inputs = self.layernorm_1(inputs)
        inputs = self.dense_1(inputs)

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=None,
            training=training,
        )
        out_1 = self.layernorm_2(inputs + attention_output_1)
        return out_1


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.embed_scale = tf.math.sqrt(tf.cast(embed_dim, tf.float32))

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_tokens = embedded_tokens * self.embed_scale
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoderBlock(layers.Layer):
    def __init__(self, embed_dim, ff_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.ffn_layer_1 = layers.Dense(ff_dim, activation="relu")
        self.ffn_layer_2 = layers.Dense(embed_dim)

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

        self.embedding = PositionalEmbedding(
            embed_dim=EMBED_DIM,
            sequence_length=SEQ_LENGTH,
            vocab_size=VOCAB_SIZE,
        )
        self.out = layers.Dense(VOCAB_SIZE, activation="softmax")

        self.dropout_1 = layers.Dropout(0.3)
        self.dropout_2 = layers.Dropout(0.5)
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, training, mask=None):
        inputs = self.embedding(inputs)
        # Instead of getting shape dynamically, use get_shape() and handle None dimensions
        input_shape = inputs.get_shape()
        batch_size = input_shape[0] if input_shape[0] is not None else tf.shape(inputs)[0]
        sequence_length = input_shape[1] if input_shape[1] is not None else tf.shape(inputs)[1]

        causal_mask = self.get_causal_attention_mask(inputs, (batch_size, sequence_length))  # Pass input_shape
        if mask is not None:
            padding_mask = tf.cast(mask[:, :, tf.newaxis], dtype=tf.int32)
            combined_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.int32)
            combined_mask = tf.minimum(combined_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=combined_mask,
            training=training,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
            training=training,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        ffn_out = self.ffn_layer_1(out_2)
        ffn_out = self.dropout_1(ffn_out, training=training)
        ffn_out = self.ffn_layer_2(ffn_out)

        ffn_out = self.layernorm_3(ffn_out + out_2, training=training)
        ffn_out = self.dropout_2(ffn_out, training=training)
        preds = self.out(ffn_out)
        return preds

    def get_causal_attention_mask(self, inputs, input_shape):
        batch_size, sequence_length = input_shape
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, sequence_length, sequence_length))
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.constant([1, 1], dtype=tf.int32),
            ],
            axis=0,
        )
        return tf.tile(mask, mult)


class ImageCaptioningModel(keras.Model):
    def __init__(
        self,
        cnn_model,
        encoder,
        decoder,
        num_captions_per_image=5,
        image_aug=None,
    ):
        super().__init__()
        self.cnn_model = cnn_model
        self.encoder = encoder
        self.decoder = decoder
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.acc_tracker = keras.metrics.Mean(name="accuracy")
        self.num_captions_per_image = num_captions_per_image
        self.image_aug = image_aug

    def calculate_loss(self, y_true, y_pred, mask):
        loss = self.loss(y_true, y_pred)
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)

    def calculate_accuracy(self, y_true, y_pred, mask):
        accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=2))
        accuracy = tf.math.logical_and(mask, accuracy)
        accuracy = tf.cast(accuracy, dtype=tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)

    def _compute_caption_loss_and_acc(self, img_embed, batch_seq, training=True):
        encoder_out = self.encoder(img_embed, training=training)
        batch_seq_inp = batch_seq[:, :-1]
        batch_seq_true = batch_seq[:, 1:]
        mask = tf.math.not_equal(batch_seq_true, 0)
        batch_seq_pred = self.decoder(
            batch_seq_inp, encoder_out, training=training, mask=mask
        )
        loss = self.calculate_loss(batch_seq_true, batch_seq_pred, mask)
        acc = self.calculate_accuracy(batch_seq_true, batch_seq_pred, mask)
        return loss, acc

    def train_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        if self.image_aug:
            batch_img = self.image_aug(batch_img)

        # 1. Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            with tf.GradientTape() as tape:
                loss, acc = self._compute_caption_loss_and_acc(
                    img_embed, batch_seq[:, i, :], training=True
                )

                # 3. Update loss and accuracy
                batch_loss += loss
                batch_acc += acc

            # 4. Get the list of all the trainable weights
            train_vars = (
                self.encoder.trainable_variables + self.decoder.trainable_variables
            )

            # 5. Get the gradients
            grads = tape.gradient(loss, train_vars)

            # 6. Update the trainable weights
            self.optimizer.apply_gradients(zip(grads, train_vars))

        # 7. Update the trackers
        batch_acc /= float(self.num_captions_per_image)
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 8. Return the loss and accuracy values
        return {
            "loss": self.loss_tracker.result(),
            "acc": self.acc_tracker.result(),
        }

    def test_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        # 1. Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            loss, acc = self._compute_caption_loss_and_acc(
                img_embed, batch_seq[:, i, :], training=False
            )

            # 3. Update batch loss and batch accuracy
            batch_loss += loss
            batch_acc += acc

        batch_acc /= float(self.num_captions_per_image)

        # 4. Update the trackers
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 5. Return the loss and accuracy values
        return {
            "loss": self.loss_tracker.result(),
            "acc": self.acc_tracker.result(),
        }

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker, self.acc_tracker]

# ... (Previous code)

# Data augmentation for image data
image_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomContrast(0.3),
    ]
)

# ... (Rest of the code)
cnn_model = get_cnn_model()
encoder = TransformerEncoderBlock(embed_dim=EMBED_DIM, dense_dim=FF_DIM, num_heads=1)
decoder = TransformerDecoderBlock(embed_dim=EMBED_DIM, ff_dim=FF_DIM, num_heads=2)
caption_model = ImageCaptioningModel(
    cnn_model=cnn_model,
    encoder=encoder,
    decoder=decoder,
    image_aug=image_augmentation,
)

In [42]:
# Define the loss function
cross_entropy = keras.losses.SparseCategoricalCrossentropy(
    from_logits=False,
    reduction=None,
)

# EarlyStopping criteria
early_stopping = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)


# Learning Rate Scheduler for the optimizer
class LRSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, post_warmup_learning_rate, warmup_steps):
        super().__init__()
        self.post_warmup_learning_rate = post_warmup_learning_rate
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        global_step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        warmup_progress = global_step / warmup_steps
        warmup_learning_rate = self.post_warmup_learning_rate * warmup_progress
        return tf.cond(
            global_step < warmup_steps,
            lambda: warmup_learning_rate,
            lambda: self.post_warmup_learning_rate,
        )


# Create a learning rate schedule
num_train_steps = len(train_dataset) * EPOCHS
num_warmup_steps = num_train_steps // 15
lr_schedule = LRSchedule(post_warmup_learning_rate=1e-4, warmup_steps=num_warmup_steps)

# Compile the model
caption_model.compile(optimizer=keras.optimizers.Adam(lr_schedule), loss=cross_entropy)

# Fit the model
caption_model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=valid_dataset,
    callbacks=[early_stopping],
)

Epoch 1/30


ValueError: as_list() is not defined on an unknown TensorShape.